In [1]:
from __future__ import division
import datetime
import matplotlib.pyplot as plt
import numpy as np
import v2gsim 
import utility
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
import os

# Create a project and initialize it with some itineraries
project = v2gsim.model.Project()
project = v2gsim.itinerary.from_excel(project, '../data/NHTS/California.xlsx')
project = v2gsim.itinerary.copy_append(project, nb_of_days_to_add=2)

# This function from the itinerary module return all the vehicles that
# start and end their day at the same location (e.g. home)
project.vehicles = v2gsim.itinerary.get_cycling_itineraries(project)

# Reduce the number of vehicles
project.vehicles = project.vehicles[0:100]

# Create some new charging infrastructures, append those new
# infrastructures to the project list of infrastructures
charging_stations = []
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2', maximum_power=7200, minimum_power=0))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L1_V1G', maximum_power=1400, minimum_power=0, post_simulation=True))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2_V2G', maximum_power=7200, minimum_power=-7200, post_simulation=True))
project.charging_stations.extend(charging_stations)

#Create a data frame with the new infrastructures mix and
# apply this mix at all the locations
df = pd.DataFrame(index=['L2', 'L1_V1G', 'L2_V2G'],
                      data={'charging_station': charging_stations,
                            'probability': [0.0, 0.4, 0.6]})
for location in project.locations:
    if location.category in ['Work', 'Home']:
        location.available_charging_station = df.copy()

# Initiate SOC and charging infrastructures
v2gsim.core.initialize_SOC(project, nb_iteration=2)

# Assign a basic result function to save power demand
for vehicle in project.vehicles:
    vehicle.result_function = v2gsim.post_simulation.netload_optimization.save_vehicle_state_for_optimization


# Launch the simulation
v2gsim.core.run(project, date_from=project.date + datetime.timedelta(days=1),
                date_to=project.date + datetime.timedelta(days=2),
                reset_charging_station=False)


# Look at the results
total_power_demand = v2gsim.post_simulation.result.total_power_demand(project)

# Optimization
myopti = v2gsim.post_simulation.netload_optimization.CentralOptimization(project, 10,
                                                                         project.date + datetime.timedelta(days=1),
                                                                         project.date + datetime.timedelta(days=2),
                                                                         minimum_SOC=0.1, maximum_SOC=0.95)

/Users/meiyewang/opt/anaconda3/envs/py27/lib/python2.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
/Users/meiyewang/opt/anaconda3/envs/py27/lib/python2.7/site-packages/v2gsim/core.py:194: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  axis=0)
core.initialize_SOC: 100%|####################################################|
core.run:  15%|#########                                                      |       mean  mean_rate       std  std_rate
0  0.950000   0.000000  0.000000  0.000000
1  0.926814   0.023186  0.110897 -0.110897
2  0.926647   0.000167  0.110849  0.000049

core.run: 100%|###############################################################|core.run: 100%|####

In [17]:
# # Load the load data
load=pd.read_csv('../data/netload/CAISO1819.csv')
startday = datetime.datetime(2018, 7, 2)#iterate through 1 day per time

load['date_time'] = [datetime.datetime.strptime(x,"%m/%d/%Y %H:%M") for x in load['date_time']]
load=load.set_index(load['date_time'])
load=load.drop(columns='date_time')

# ####################################### add the component of loop around day
load = pd.DataFrame(load[startday: startday +datetime.timedelta(days=1)]['demand (MW)'])

# load = pd.DataFrame(load[startday]['demand (MW)'])
# load

,demand (MW)
date_time,
2018-07-02 00:00:00,31187
2018-07-02 01:00:00,32303
2018-07-02 02:00:00,32870
2018-07-02 03:00:00,32478
2018-07-02 04:00:00,31983
2018-07-02 05:00:00,30949
2018-07-02 06:00:00,28655
2018-07-02 07:00:00,26236
2018-07-02 08:00:00,24304


In [16]:
price_pd=getprice(instance,cost_pd)

myresult = myopti.solve(project, load * 1000000,1500000, price=price_pd,
                    peak_shaving='economic', SOC_margin=0.05)

# Get the result in the right format
temp_vehicle = pd.DataFrame(
    (total_power_demand['total'] - myresult['vehicle_before'] + myresult['vehicle_after']) *
    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle = temp_vehicle.rename(columns={0: 'vehicle'})
temp_vehicle['index'] = range(0, len(temp_vehicle))
temp_vehicle = temp_vehicle.set_index(['index'], drop=True)

temp_load = load.copy()
temp_load = temp_load.resample('60S')
temp_load = temp_load.fillna (method='ffill').fillna(method='bfill')
temp_load = temp_load.head(len(temp_vehicle))
tempIndex = temp_load.index
temp_load['index'] = range(0, len(temp_vehicle))
temp_load = temp_load.set_index(['index'], drop=True)

temp_result = pd.DataFrame(temp_load['demand (MW)'] + temp_vehicle['vehicle'])
temp_result = temp_result.rename(columns={0: 'demand (MW)'})
temp_result = temp_result.set_index(tempIndex)
temp_load = temp_load.set_index(tempIndex)
temp_vehicle = temp_vehicle.set_index(tempIndex)

temp_vehicle_demand = pd.DataFrame(
    (myresult['vehicle_after_demand']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_demand = temp_vehicle_demand.set_index(tempIndex)

temp_vehicle_generation = pd.DataFrame(
    (-myresult['vehicle_after_generation']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_generation = temp_vehicle_generation.set_index(tempIndex)

temp_vehicle_regdown = pd.DataFrame(
    (myresult['vehicle_after_regdown']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_demand = temp_vehicle_demand.set_index(tempIndex)

temp_vehicle_regup = pd.DataFrame(
    (-myresult['vehicle_after_regup']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_generation = temp_vehicle_generation.set_index(tempIndex)


plt.plot(temp_load['demand (MW)'], label='load')
plt.plot(temp_result['demand (MW)'], label='load + vehicles')
plt.ylabel('Power (MW)')
plt.legend()
plt.show()

NameError: name 'getprice' is not defined